# An Introduction to LangChain

In this notebook, we will cover the very basics of LangChain. This includes chains and templates.

First, we import necessary libraries, and use `dotenv` to load our OpenAI API key.

In [1]:
from langchain_community.chat_models import ChatOpenAI
import openai
import os

from dotenv import load_dotenv

api_key = os.getenv("OPENAI_API_KEY")

load_dotenv()

True

In [2]:
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)

/var/folders/tl/wt0ch7xx1vbfcytgzjpnnsy40000gn/T/ipykernel_11943/1543617194.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)


In every chat, LLMs are typically first introduced to a "system message," instructing the LLM on how to interpret the conversation. There is also a "human" or "user" message which is simply what the user sends to the LLM. An "assistant" or "AI" message is associated with the messages that the LLM itself writes. 

In [4]:
messages = [
    SystemMessage(content="You are an expert data scientist."),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data. Only return the script.")
]

response = llm(messages=messages, temperature=0.6)
print(response.content)

/var/folders/tl/wt0ch7xx1vbfcytgzjpnnsy40000gn/T/ipykernel_11943/2497210350.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages=messages, temperature=0.6)


```python
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, size=(1000, 1))

# Define the neural network
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)
```


Note the use of the "temperature" parameter. Temperature is a parameter that refers to the probability with which the LLM's underlying next token predictor picks out a next token that is not the highest probability token. You can consider to it a proxy for "creativity." Higher temperature -> more randomness -> more "creativity."

We represent prompts using "prompt templates," which allow us to dynamically plug things into prompts. The `PromptTemplate` class is simply LangChain's object interface with prompts. Chains allow us to link prompts together. 

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [6]:
prompt = """
Explain {topic} in one sentence
"""

prompt = PromptTemplate.from_template(prompt)

In [7]:
chain = LLMChain(prompt=prompt, llm=llm)

chain.run(topic="atitude")

/var/folders/tl/wt0ch7xx1vbfcytgzjpnnsy40000gn/T/ipykernel_11943/1723417862.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm)
/var/folders/tl/wt0ch7xx1vbfcytgzjpnnsy40000gn/T/ipykernel_11943/1723417862.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(topic="atitude")


"Attitude refers to a person's overall outlook or disposition towards a particular situation, person, or event."

In [8]:
second_prompt = PromptTemplate(
    input_variables=["ml_topic_desc"],
    template="""
    You are a:
    {ml_topic_desc}
    """
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain.run(topic = "potato")

'Potato is a starchy tuber vegetable that is a staple food in many cuisines around the world.'

The outputs of the first chain are passed in to the second chain as input.

In [9]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

blog_post = overall_chain.run(input="autoencoder")
print(blog_post)



> Entering new SimpleSequentialChain chain...
An autoencoder is a type of neural network that learns to encode input data into a lower-dimensional representation and then decode it back to its original form.

Autoencoders are commonly used for tasks such as data compression, denoising, and dimensionality reduction. They consist of an encoder and a decoder, which are typically implemented as neural networks. The encoder takes the input data and generates a compressed representation (also known as a latent space representation) of the data. The decoder then takes this compressed representation and reconstructs the original input data.

Autoencoders are unsupervised learning models, meaning they do not require labeled data for training. Instead, they learn to reconstruct the input data by minimizing a reconstruction error, such as mean squared error or binary cross-entropy loss.

Autoencoders have been used in various applications, such as image and video processing, anomaly detection, 